In [ ]:
%pip install tqdm requests

In [1]:
import os
import requests
from tqdm.notebook import tqdm
from pathlib import Path

In [2]:
def create_folder(path):
    Path(path).mkdir(parents=True, exist_ok=True)

def download_image(url, path):
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            with open(path, 'wb') as f:
                f.write(response.content)
            return True
    except Exception as e:
        pass
    return False

In [3]:
def get_pokemon_data(start: int, end: int):
    url = "https://pokeapi.co/api/v2/pokemon?limit=10000"
    response = requests.get(url)
    data = response.json()
    
    pokemon = []
    for entry in data['results']:
        name = entry['name']
        try:
            url_id = int(entry['url'].split('/')[-2])
        except:
            continue
        if start <= url_id <= end:
            pokemon.append((name, url_id))
    return pokemon


In [4]:
def check_image_exists(url: str) -> bool:
    #print(url)
    """Verificar se existe a imagem na URL fornecida"""
    try:
        response = requests.head(url, allow_redirects=True)
        return response.status_code == 200
    except requests.RequestException:
        return False

In [6]:
def baixar_pokemon(pokemon_name: str, pokemon_id: int, folder_path: str):
    forms = ["", "-mega", "-megax", "-megay", "-gmax", "-alola", "-galar", "-hisui", "-paldea"]
    
    # Formatar o nome da pasta como "XXXX - Nome do pokémon"
    folder_name = f"{str(pokemon_id).zfill(4)} - {pokemon_name.capitalize()}"
    folder = Path(folder_path) / folder_name
    folder.mkdir(parents=True, exist_ok=True)
    
    base_name = pokemon_name.lower().replace(".","").replace(" ","").replace("-","")
    base_id = str(pokemon_id).zfill(3)
    base_url = f"https://assets.pokemon.com/assets/cms2/img/pokedex/full/"
    
    for form in forms:
        form_name = base_name + form
        
        # PNGs (oficial e shiny)
        if form == "":
            # Oficial
            png_url = f"{base_url}{base_id}.png"
            #print(png_url)
            
            if not (check_image_exists(png_url)):
                continue
            
            download_image(png_url, folder / f"{form_name}.png")
            
            # Verificar formas em PNG
            suffix = 2
            while True:
                variation_url = f"{base_url}{base_id}_f{suffix}.png"
                success = download_image(variation_url, folder / f"{form_name}_f{suffix}.png")
                if not success:
                    break
                suffix += 1
        else:
            # Sprites alternativos que não existem no site oficial
            gif_url = f"https://play.pokemonshowdown.com/sprites/ani/{form_name}.gif"
            
            if not (check_image_exists(gif_url)):
                continue
                
            #shiny_sprite_url = f"https://play.pokemonshowdown.com/sprites/ani-shiny/{form_name}.gif"
            #download_image(gif_url, folder / f"{form_name}.png")
            #download_image(shiny_sprite_url, folder / f"{form_name}-shiny.png")
        
        # GIFs (normais e shiny)
        gif_urls = [
            f"https://play.pokemonshowdown.com/sprites/ani/{form_name}.gif",
            f"https://play.pokemonshowdown.com/sprites/gen5ani/{form_name}.gif",
            f"https://play.pokemonshowdown.com/sprites/gen5/{form_name}.png",            
        ]
        shiny_gif_urls = [
            f"https://play.pokemonshowdown.com/sprites/ani-shiny/{form_name}.gif",
            f"https://play.pokemonshowdown.com/sprites/gen5ani-shiny/{form_name}.gif",
            f"https://play.pokemonshowdown.com/sprites/gen5-shiny/{form_name}.png",            
        ]
        
        for gif_url in gif_urls:
            if check_image_exists(gif_url):
                download_image(gif_url, folder / f"{form_name}.gif")
                break
        
        for shiny_gif_url in shiny_gif_urls:
            if check_image_exists(shiny_gif_url):
                download_image(shiny_gif_url, folder / f"{form_name}-shiny.gif")
                break


In [8]:
# Caminho de destino
#base_folder = r"C:\Users\corps\Documents\Cursor\github\Pokedex\jupyter\images"
base_folder = r"C:\Users\artur\Documents\Pokedex\jupyter\images"
create_folder(base_folder)

# Faixa da Pokédex
start_id = int(input("Número inicial da Pokédex: "))
end_id = int(input("Número final da Pokédex: "))

# Obter lista
lista_pokemon = get_pokemon_data(start_id, end_id)

# Baixar todos
for name, pid in tqdm(lista_pokemon):
    baixar_pokemon(name, pid, base_folder)


PermissionError: [WinError 5] Acesso negado: 'C:\\Users\\artur'

In [11]:
# Verificar nomes de arquivos errados
def verify_shiny(pasta: str):
    pasta = Path(pasta)
    arquivos_errados = []

    for arquivo in pasta.rglob("*.gif"):
        if not arquivo.is_file():
            continue
        
        nome = arquivo.stem.lower()

        if "shiny" in nome and not nome.endswith("-shiny"):
            arquivos_errados.append(arquivo)

    if arquivos_errados:
        print("Arquivos errados:\n")
        for arq in arquivos_errados:
            print(arq)
    else:
        print("Nenhum arquivo nomeado errado!")

verify_shiny(r"C:\Users\corps\Documents\Cursor\github\Pokedex\jupyter\images")
#verify_shiny(r"C:\Users\artur\Documents\Pokedex\jupyter\images")

Nenhum arquivo nomeado errado!


In [12]:
from pathlib import Path

def verificar_variantes(pasta: str):
    pasta = Path(pasta)
    # { base_lower: { 'png': bool, 'gif': bool, 'shiny': bool } }
    formatos = {}

    for arq in pasta.rglob("*.*"):
        if not arq.is_file():
            continue
        ext = arq.suffix.lower()
        if ext not in (".png", ".gif"):
            continue

        stem = arq.stem           # ex: 'charizard', ou 'charizard-shiny'
        stem_lower = stem.lower()

        # detecta o '-shiny' removendo o sufixo
        if ext == ".gif" and stem_lower.endswith("-shiny"):
            base = stem_lower[:-6]  # tira o "-shiny"
            formatos.setdefault(base, {"png": False, "gif": False, "shiny": False})
            formatos[base]["shiny"] = True
        else:
            base = stem_lower
            formatos.setdefault(base, {"png": False, "gif": False, "shiny": False})
            if ext == ".png":
                formatos[base]["png"] = True
            elif ext == ".gif":
                formatos[base]["gif"] = True

    # agora monta a lista de faltantes
    faltantes = {
        base: [ext for ext, existe in flags.items() if not existe]
        for base, flags in formatos.items()
        if any(not existe for existe in flags.values())
    }

    if faltantes:
        print("Bases com variantes faltando:")
        for base, miss in sorted(faltantes.items()):
            print(f"  • {base}: faltando {', '.join(miss)}")
    else:
        print("Todas as bases possuem .png, .gif e -shiny.gif.")


# Exemplo de uso:
verificar_variantes(r"C:\Users\corps\Documents\Cursor\github\Pokedex\jupyter\images")
#verificar_variantes(r"C:\Users\artur\Documents\Pokedex\jupyter\images")

Bases com variantes faltando:
  • arceus-bug: faltando png
  • arceus-dark: faltando png
  • arceus-dragon: faltando png
  • arceus-electric: faltando png
  • arceus-fairy: faltando png
  • arceus-fighting: faltando png
  • arceus-fire: faltando png
  • arceus-flying: faltando png
  • arceus-ghost: faltando png
  • arceus-grass: faltando png
  • arceus-ground: faltando png
  • arceus-ice: faltando png
  • arceus-poison: faltando png
  • arceus-psychic: faltando png
  • arceus-rock: faltando png
  • arceus-steel: faltando png
  • arceus-water: faltando png
  • blastoise-gmax: faltando gif, shiny
  • cinderace-gmax: faltando gif, shiny
  • dudunsparce: faltando png
  • dudunsparce-twosegment: faltando gif, shiny
  • eiscue-noice: faltando png
  • furfrou-dandy: faltando png
  • furfrou-debutante: faltando png
  • furfrou-kabuki: faltando png
  • furfrou-lareine: faltando png
  • furfrou-matron: faltando png
  • furfrou-pharaoh: faltando png
  • genesect-burn: faltando png
  • genesect-ch

c:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


AttributeError: 'list' object has no attribute 'astype'